Importo librerie e definisco costanti

In [ ]:
import random
import datetime
import copy # per copiare gli oggetti per valore e non per riferimento
import pandas
import csv
import matplotlib.pyplot as pyplot

# Dati per simulazione di un giorno
SEP = "-"*60
MIN_GRP = 2
MAX_GRP = 10
N_BARCHE = 5
N_ESCURSIONI = 5
N_UTENTI = random.randrange(50,100)

Dati per statistiche

In [ ]:
n_prenotazioni_ok = 0
n_prenotazioni_fail = 0
n_persone_ok = 0
n_persone_fail = 0
n_prenotazioni_ok_con_riall = 0
n_persone_ok_con_riall = 0

Classe di utilità per i log delle prenotazioni

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

Definisco la classe Barca

In [ ]:
class Barca:
  def __init__(self, modello , n_posti):
    super().__init__()
    self.modello = modello
    self.__n_posti = n_posti
    self.__n_posti_occupati = 0
    # utenti attualmente sulla barca
    self.__utenti_assegnati = []

  def to_string(self):
    print("{} {} posti".format(self.modello, self.__n_posti))

  def get_n_posti(self):
    return self.__n_posti
  
  def get_posti_rimanenti(self):
    return self.__n_posti-self.__n_posti_occupati 
    
  def assegna_utente(self, utente, gruppo_diviso=None):
    n = 0
    if (gruppo_diviso is not None):
      if (gruppo_diviso <= 0):
        raise Exception("Errore! Hai indicato un gruppo diviso errato")
      else:
        n = gruppo_diviso
    else:
      n = utente.n_persone
      
    if (n <= self.get_posti_rimanenti()):
      self.__n_posti_occupati+=n
      self.__utenti_assegnati.append(utente)
    else:
      raise Exception("Posti non sufficienti")

  def get_utenti_assegnati(self):
    return self.__utenti_assegnati

Creo un DB di barche, ordinate per numero di posti

In [ ]:
db_barche = []
for i in range(N_BARCHE):
  db_barche.append(Barca("B{}".format(i+1), i+11)) # già ordinate in ordine di n_posti
  db_barche[i].to_string()

Definisco la classe Utente

In [ ]:
class Utente:
  def __init__(self, nome , n_persone):
    super().__init__()
    self.nome = nome
    self.n_persone = n_persone
  
  def to_string(self):
    print("{}\t {} persone".format(self.nome, self.n_persone))

Definisco la classe Escursione, questa utilizza il DB di barche creato prima

In [ ]:
class Escursione:
  counter = 0

  def __init__(self, data):
    super().__init__()
    Escursione.counter +=1 
    self.index = Escursione.counter
    self.data = data
    self.barche_disponibili = copy.deepcopy(db_barche) # array di barche, effettuo una COPIA del db, non modifico il db (per ora faccio così, poi dovresti gestire il tempo!)

  def get_posti_rimanenti_tot(self):
    res = 0
    for barca in self.barche_disponibili:
      res+=barca.get_posti_rimanenti()
    return res

  def to_string(self):
    print("Escursione {} del {}".format(self.index, self.data))
    for barca in self.barche_disponibili:
      s = ""
      utenti = barca.get_utenti_assegnati() 
      if (len(utenti)!=0):
        for utente in utenti:
          s+=utente.nome + " "
      print("{} ({}/{} posti rimanenti) {}".format(barca.modello, barca.get_posti_rimanenti(), barca.get_n_posti(), s))


Definisco la classe Prenotazione

In [ ]:
class Prenotazione:
  def __init__(self, utente, escursione):
    super().__init__()
    self.utente = utente
    self.escursione = escursione
    self.barca = None
    self.barca_secondaria = None
  
  def to_string(self):
    if (self.barca is not None and self.barca_secondaria is None):
      print(color.BOLD + color.GREEN + "Prenotazione effettuata: {} in {} ({}/{})".format(self.utente.nome, self.barca.modello, self.barca.get_posti_rimanenti(), self.barca.get_n_posti()) + color.END)
    elif(self.barca is not None and self.barca_secondaria is not None):
      print(color.BOLD + color.BLUE + "Prenotazione effettuata: {} in {} ({}/{}) e in {} ({}/{})".format(self.utente.nome, self.barca.modello, self.barca.get_posti_rimanenti(), self.barca.get_n_posti(), self.barca_secondaria.modello, self.barca_secondaria.get_posti_rimanenti(), self.barca_secondaria.get_n_posti()) + color.END)
    else:
      print(color.BOLD + color.RED + "Prenotazione fallita!" + color.END)


Creo un DB di utenti con un loro gruppo di persone

In [ ]:
utenti = []
for i in range(N_UTENTI):
  utenti.append(Utente("U{}".format(i+1), random.randrange(MIN_GRP, MAX_GRP)))
  utenti[i].to_string()

Creo un DB di escursioni, rendendole disponibili per la prenotazione

In [ ]:
print(SEP)
escursioni = []
for i in range(N_ESCURSIONI):
  escursioni.append(Escursione(datetime.datetime(2021,random.randrange(1,12),random.randrange(1,28), random.randrange(8,18))))
  escursioni[i].to_string()
  print(SEP)

Definisco la funzione che prende in ingresso la prenotazione e ne modifica il campo relativo alla barca scelta


In [ ]:
def assegna_barca(prenotazione):
  global n_prenotazioni_ok
  global n_persone_ok
  global n_prenotazioni_fail
  global n_persone_fail
  global n_prenotazioni_ok_con_riall
  global n_persone_ok_con_riall

  print("Algoritmo avviato")
  n = prenotazione.escursione.get_posti_rimanenti_tot()
  n_persone = prenotazione.utente.n_persone
  if (n>0):
    print("{} posti totali rimanenti per l'escursione".format(n))
    barca_ha_abbasta_posti_rimanenti = lambda barca: (barca.get_posti_rimanenti()>=n_persone)
    barche_con_abbasta_posti_rimanenti = list(filter(barca_ha_abbasta_posti_rimanenti, prenotazione.escursione.barche_disponibili))
    if (len(barche_con_abbasta_posti_rimanenti)>0):
      # esiste una barca (vuota o non vuota) in grado di contenere il gruppo 
      is_barca_occupata = lambda barca: (barca.get_n_posti() > barca.get_posti_rimanenti()) # f(barca)-> true se non è vuota
      barche_occupate = list(filter(is_barca_occupata, barche_con_abbasta_posti_rimanenti))
      if (len(barche_occupate)>0):
        print("Scelta barca non vuota")
        barca_scelta = barche_occupate[0] # 0 = prima barca (più piccola), -1 = ultima barca (più grande) 
      else:
        print("Scelta barca vuota")
        # cerco la più piccola tra le barche vuote
        is_barca_vuota = lambda barca: (barca.get_n_posti() == barca.get_posti_rimanenti()) # f(barca)-> true se è vuota
        barche_vuote = list(filter(is_barca_vuota, barche_con_abbasta_posti_rimanenti))
        barca_scelta = barche_vuote[0] # 0 = prima barca (più piccola), -1 = ultima barca (più grande)
        # barca_scelta = barche_vuote[len(barche_occupate)-1] # la prima barca è quella più grande
      
      # Ora aggiorno i campi della prenotazione
      prenotazione.barca = barca_scelta
      indice_barca_scelta = prenotazione.escursione.barche_disponibili.index(barca_scelta)
      prenotazione.escursione.barche_disponibili[indice_barca_scelta].assegna_utente(prenotazione.utente)

      n_prenotazioni_ok +=1
      n_persone_ok += n_persone
    else:
      riallocazione_fatta = False
      if (n_persone>=4):
        print("Provo a dividere il gruppo e ad allocarlo in 2 barche diverse")
        n_persone_gruppo_2 = int(n_persone/2)
        n_persone_gruppo_1 = 0
        if (n_persone%2==0):
          n_persone_gruppo_1 = n_persone_gruppo_2
        else:
          n_persone_gruppo_1 = int(n_persone/2+1)

        barche_con_abbasta_posti_rimanenti = list(filter(lambda barca: (barca.get_posti_rimanenti()>=n_persone_gruppo_1), prenotazione.escursione.barche_disponibili))
        # esistono 2 barche in grado di contenere i due gruppi        
        if (len(barche_con_abbasta_posti_rimanenti)>1):
          barca_scelta1 = barche_con_abbasta_posti_rimanenti[0]
          barca_scelta2 = barche_con_abbasta_posti_rimanenti[1]

          # Ora aggiorno i campi della prenotazione
          prenotazione.barca = barca_scelta1
          prenotazione.barca_secondaria = barca_scelta2
          indice_barca_scelta1 = prenotazione.escursione.barche_disponibili.index(barca_scelta1)
          indice_barca_scelta2 = prenotazione.escursione.barche_disponibili.index(barca_scelta2)
          
          prenotazione.escursione.barche_disponibili[indice_barca_scelta1].assegna_utente(prenotazione.utente, n_persone_gruppo_1)
          prenotazione.escursione.barche_disponibili[indice_barca_scelta2].assegna_utente(prenotazione.utente, n_persone_gruppo_2)
          
          riallocazione_fatta = True
        else:
          print("OPS! Ci sono ancora posti, ma sono sparsi per le barche e anche dividendo il gruppo non sono riuscito ad allocarlo.")
      else:
        print("OPS! Ci sono ancora posti, ma sono sparsi per le barche e il gruppo è troppo piccolo per essere diviso")

      if (riallocazione_fatta):
        n_prenotazioni_ok_con_riall +=1
        n_persone_ok_con_riall += n_persone
      else: 
        n_prenotazioni_fail +=1
        n_persone_fail += n_persone
  else:
    print("OPS! Posti finiti!") 
    n_prenotazioni_fail +=1
    n_persone_fail += n_persone


Simulo la prenotazione per ogni gruppo per un'escursione casuale


In [ ]:
prenotazioni = []
for i in range(len(utenti)):
  print(SEP)
  indice_escursione_scelta = random.randrange(0, N_ESCURSIONI)
  print("{} sta prenotando per {} persone per l'escursione {}".format(utenti[i].nome, utenti[i].n_persone, escursioni[indice_escursione_scelta].index))
  p = Prenotazione(utenti[i], escursioni[indice_escursione_scelta]) # creo prenotazione
  assegna_barca(p) # eseguo l'algoritmo sulla prenotazione (0 scelgo sempre barca più piccola, -1 sempre quella grande)
  p.to_string()
  prenotazioni.append(p)
print(SEP)

Stato barche post prenotazioni

In [ ]:
for i in range(N_ESCURSIONI):
  escursioni[i].to_string()
  print()

Report prenotazioni

In [ ]:
n_prenotazioni_tot = n_prenotazioni_ok + n_prenotazioni_fail + n_prenotazioni_ok_con_riall
print("Prenotazioni: {}/{}".format(n_prenotazioni_ok+n_prenotazioni_ok_con_riall, n_prenotazioni_tot))

n_persone_tot = n_persone_ok + n_persone_fail + n_persone_ok_con_riall 
print("Persone {}/{}".format(n_persone_ok+n_persone_ok_con_riall, n_persone_tot))

Salvo tutti i risultati in un file csv

In [ ]:
file_path = "./file.csv"
col_names = ["Prenotazione normale", "Prenotazione divisa", "Fail", "Gruppo unito", "Gruppi divisi", "Non inserite"]
df = pandas.DataFrame([[n_prenotazioni_ok, n_prenotazioni_ok_con_riall, n_prenotazioni_fail,n_persone_ok, n_persone_ok_con_riall, n_persone_fail]], columns = col_names)
df.to_csv(file_path, mode='a', header=None, index=False)


Mostro risultati totali

In [ ]:
df = pandas.read_csv(file_path, names=col_names)
file_csv = pandas.read_csv(file_path, names=col_names)
mycolors = ["green", "blue", "red"]

prenotazioni_ok_tot = file_csv[col_names[0]].sum()
prenotazioni_ok_con_riall_tot = file_csv[col_names[1]].sum()
prenotazioni_fail_tot = file_csv[col_names[2]].sum()
pyplot.title("Prenotazioni")
pyplot.pie([prenotazioni_ok_tot, prenotazioni_ok_con_riall_tot, prenotazioni_fail_tot], labels=col_names[:3], colors=mycolors)
pyplot.show()

persone_ok_tot = file_csv[col_names[3]].sum()
persone_ok_con_riall_tot = file_csv[col_names[4]].sum()
persone_fail_tot = file_csv[col_names[5]].sum()
pyplot.title("Persone")
pyplot.pie([persone_ok_tot, persone_ok_con_riall_tot, persone_fail_tot], labels=col_names[3:6], colors=mycolors)
pyplot.show()

